## Data Wrangling - Zipcode Aggregation of All Datas
from FRED public data and Zillow public data

In [1]:
#Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

In [2]:
#change directory to get data
path= '/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim'
os.chdir(path)

In [3]:
os.listdir()

['.gitkeep',
 '.ipynb_checkpoints',
 'ACS_rentsPrices_2011_2018.csv',
 'ACS_zillow_2011_2018.csv',
 'Annual_fredData_2011_2019',
 'df3_1956',
 'df_homePrices_2011_2018',
 'df_homePrices_2011_2020',
 'df_homePrices_2011_2020.csv',
 'df_rentPrices_2014_2020',
 'df_rentPrices_2014_2020.csv',
 'master_complete_for_EDA.csv',
 'VacancyRate_Zipcode_2011_2018',
 'VacancyRate_Zipcode_2011_2018.csv',
 'VacancyRate_Zipcode_AND_National_2011_2020.csv',
 'zillow_rentPrices_2014_2020.csv']

In [4]:
#load rental data
df_fred = pd.read_csv('Annual_fredData_2011_2019')
df_homes = pd.read_csv("df_homePrices_2011_2020.csv", converters={'RegionName': lambda x: str(x)})
df_rents = pd.read_csv('ACS_rentsPrices_2011_2018.csv', dtype={'RegionID': object, 'RegionName': object})
df_vacancy = pd.read_csv('VacancyRate_Zipcode_2011_2018.csv', dtype={'Zipcode': object})

#secondary rental data
ACS_Rents2011_18 = pd.read_csv('ACS_zillow_2011_2018.csv', dtype={'RegionID': object, 'RegionName': object})
zillow_Rents2014_20 = pd.read_csv('zillow_rentPrices_2014_2020.csv', dtype={'RegionID': object, 'RegionName': object})

### Data Definition

In [5]:
df_vacancy

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%,Year
0,02333,3.024027,2.199925,2011
1,02338,3.116343,2.948791,2011
2,02339,4.464646,2.066438,2011
3,02341,3.586322,2.340722,2011
4,02343,3.732901,2.926524,2011
...,...,...,...,...
264955,98279,51.219512,10.993457,2018
264956,98280,51.329243,12.777549,2018
264957,98311,6.540162,1.960476,2018
264958,98326,28.537736,14.679524,2018


In [6]:
df_homes

,SizeRank,RegionName,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
303665,34430,43523,OH,Malinta,NaN,Henry County,84331.11,2020
303666,34430,7703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,579932.56,2020
303667,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,42896.56,2020
303668,35187,822,LA,Choudrant,Ruston,Lincoln Parish,195888.67,2020


### Data Cleaning 1 - HomePrices

In [7]:
#add leading zeros to zipcode in home dataframe
df_homes['RegionName'] = df_homes['RegionName'].apply(lambda x: '{0:0>5}'.format(x))

In [8]:
df_homes

,SizeRank,RegionName,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
303665,34430,43523,OH,Malinta,NaN,Henry County,84331.11,2020
303666,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,579932.56,2020
303667,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,42896.56,2020
303668,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,195888.67,2020


In [9]:
#rename Zipcode column
df_homes = df_homes.rename(columns = {'RegionName':'Zipcode'})
#check for NaNs
df_homes.isna().sum()

SizeRank          0
Zipcode           0
State             0
City              0
Metro         71180
CountyName        0
HomePrice     11066
Year              0
dtype: int64

In [10]:
#subset df_homes data for year 2011-2018 (for merging with FRED data)
df_homes_2011_2018 = df_homes.loc[df_homes['Year'] < 2019]

In [11]:
#subset df_homes data for year 2014-2020 (for merging with rent data)
df_homes_2014_2020 = df_homes.loc[df_homes['Year'] > 2013]

In [12]:
df_homes_2014_2020[df_homes_2014_2020.Year == 2014]

,SizeRank,Zipcode,State,City,Metro,CountyName,HomePrice,Year
91101,0,10025,NY,New York,New York-Newark-Jersey City,New York County,968761.75,2014
91102,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,2014
91103,2,10023,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,2014
91104,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,2014
91105,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,2014
...,...,...,...,...,...,...,...,...
121463,34430,43523,OH,Malinta,NaN,Henry County,67959.33,2014
121464,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,NaN,2014
121465,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,NaN,2014
121466,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,140783.33,2014


In [13]:
df_homes_2011_2018

,SizeRank,Zipcode,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
242931,34430,43523,OH,Malinta,NaN,Henry County,81699.58,2018
242932,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,543333.30,2018
242933,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,40215.62,2018
242934,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,178884.17,2018


In [14]:
#check %NaNs in home_price data
df_homes_2011_2018.isna().sum()/len(df_homes_2011_2018)*100

SizeRank       0.000000
Zipcode        0.000000
State          0.000000
City           0.000000
Metro         23.439918
CountyName     0.000000
HomePrice      4.555109
Year           0.000000
dtype: float64

In [15]:
df_homes_2014_2020.isna().sum()

SizeRank          0
Zipcode           0
State             0
City              0
Metro         49826
CountyName        0
HomePrice      3340
Year              0
dtype: int64

In [16]:
#look at which rows have NaNs for homeprice 
#need to get grammar right here...
df_homes_2014_2020.loc[:, df_homes_2014_2020.isna().any()]

,Metro,HomePrice
91101,New York-Newark-Jersey City,968761.75
91102,Chicago-Naperville-Elgin,450755.75
91103,New York-Newark-Jersey City,1024543.17
91104,Houston-The Woodlands-Sugar Land,322032.00
91105,Chicago-Naperville-Elgin,580250.92
...,...,...
303665,NaN,84331.11
303666,New York-Newark-Jersey City,579932.56
303667,Terre Haute,42896.56
303668,Ruston,195888.67


In [17]:
#replace home price NaNs with first:
    #diff = (price of year you have) - (mean of a year you have) 
    #then replace NaN with 'diff' + (mean of NaN year)

### Data Cleaning 2 - RentPrices

In [18]:
df_rents

,RegionName,RentPrice,Year
0,02333,1368.536,2011
1,02338,1311.076,2011
2,02339,1484.626,2011
3,02341,1266.816,2011
4,02343,1524.006,2011
...,...,...,...
264955,98279,1059.870,2018
264956,98280,993.850,2018
264957,98311,1533.500,2018
264958,98326,778.990,2018


In [19]:
#add leading zeros to zipcode in rent dataframe
df_rents['RegionName'] = df_rents['RegionName'].apply(lambda x: '{0:0>5}'.format(x))

In [20]:
df_rents

,RegionName,RentPrice,Year
0,02333,1368.536,2011
1,02338,1311.076,2011
2,02339,1484.626,2011
3,02341,1266.816,2011
4,02343,1524.006,2011
...,...,...,...
264955,98279,1059.870,2018
264956,98280,993.850,2018
264957,98311,1533.500,2018
264958,98326,778.990,2018


In [21]:
zillow_Rents2014_20

,RegionName,RentPrice,Year
0,10025,3041.83,2014
1,60657,1589.42,2014
2,10023,3186.67,2014
3,77494,1807.33,2014
4,60614,1786.25,2014
...,...,...,...
22696,02110,4408.57,2020
22697,20004,2505.56,2020
22698,80951,1647.88,2020
22699,11964,15800.50,2020


In [22]:
#rename Zipcode column
df_rents = df_rents.rename(columns={'RegionName': 'Zipcode'})
zillow_Rents2014_20 = zillow_Rents2014_20.rename(columns={'RegionName': 'Zipcode'})

In [23]:
#check for % NaNs
df_rents.isna().sum()/len(df_rents)*100

Zipcode      0.00000
RentPrice    7.51321
Year         0.00000
dtype: float64

In [24]:
zillow_Rents2014_20.isna().sum()/len(df_rents)*100

Zipcode      0.000000
RentPrice    0.001132
Year         0.000000
dtype: float64

In [25]:
#replace home price NaNs with first:
    #diff = (price of year you have) - (mean of a year you have) 
    #then replace NaN with 'diff' + (mean of NaN year)

In [26]:
zillow_Rents2014_20

,Zipcode,RentPrice,Year
0,10025,3041.83,2014
1,60657,1589.42,2014
2,10023,3186.67,2014
3,77494,1807.33,2014
4,60614,1786.25,2014
...,...,...,...
22696,02110,4408.57,2020
22697,20004,2505.56,2020
22698,80951,1647.88,2020
22699,11964,15800.50,2020


### Data Cleaning 3 - FRED Economic Data

In [27]:
df_fred.head()

,DATE,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,2011-01-01,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,2012-01-01,0.750000,56912.0,0.727269,8.075000,783.750000,278995.583333
2,2013-01-01,0.750000,58904.0,0.686773,7.358333,928.166667,335207.333333
3,2014-01-01,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333
4,2015-01-01,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333


In [28]:
#change year to match df_vacancy format
df_fred.DATE = df_fred.DATE.str.replace('-01-01','')

In [29]:
#change DATE column name to Year
df_fred = df_fred.rename(columns = {'DATE':'Year'})

In [30]:
#subset FRED data to 2011-2018 to prep for merge
df_fred_2011_2018 = df_fred.iloc[0:8]
#change year to int for merge 
df_fred_2011_2018 = df_fred_2011_2018.astype({'Year': 'int32'})
df_fred = df_fred.astype({'Year': 'int32'})

In [31]:
df_fred

,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,2012,0.750000,56912.0,0.727269,8.075000,783.750000,278995.583333
2,2013,0.750000,58904.0,0.686773,7.358333,928.166667,335207.333333
3,2014,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333
4,2015,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333
5,2016,1.020833,62898.0,0.716669,4.875000,1176.583333,485822.500000
6,2017,1.625000,63761.0,0.631008,4.341667,1207.416667,546020.166667
7,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
8,2019,2.750000,68703.0,0.473954,3.666667,1295.250000,550386.500000


In [32]:
df_vacancy.dtypes

Zipcode              object
Vacancy_Rate%       float64
MOE-VacancyRate%    float64
Year                  int64
dtype: object

In [33]:
#subset fred data for 2014-208 for zillow merge
df_fred_2014_2019 = df_fred.iloc[3:9]
df_fred_2014_2019 

,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
3,2014,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333
4,2015,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333
5,2016,1.020833,62898.0,0.716669,4.875000,1176.583333,485822.500000
6,2017,1.625000,63761.0,0.631008,4.341667,1207.416667,546020.166667
7,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
8,2019,2.750000,68703.0,0.473954,3.666667,1295.250000,550386.500000


In [34]:
#add zipcode column to 2014-2019 with zipcode '99999' which denotes National data
df_fred_2014_2019['Zipcode'] = 99999
df_fred_2014_2019

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending,Zipcode
3,2014,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333,99999
4,2015,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333,99999
5,2016,1.020833,62898.0,0.716669,4.875000,1176.583333,485822.500000,99999
6,2017,1.625000,63761.0,0.631008,4.341667,1207.416667,546020.166667,99999
7,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000,99999
8,2019,2.750000,68703.0,0.473954,3.666667,1295.250000,550386.500000,99999


### Data Joining

In [35]:
#merge rent prices and home prices (2011-2018)
df_rent_merged_homes = pd.merge(df_rents, df_homes_2011_2018,how='left', on=['Year', 'Zipcode'])
df_rent_merged_homes

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice
0,02333,1368.536,2011,8782.0,MA,East Bridgewater,Boston-Cambridge-Newton,Plymouth County,NaN
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50
3,02341,1266.816,2011,10079.0,MA,Hanson,Boston-Cambridge-Newton,Plymouth County,NaN
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42
...,...,...,...,...,...,...,...,...,...
264955,98279,1059.870,2018,23400.0,WA,Olga,NaN,San Juan County,552805.42
264956,98280,993.850,2018,25265.0,WA,Eastsound,NaN,San Juan County,678499.00
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17


In [36]:
zillow_Rents2014_20

,Zipcode,RentPrice,Year
0,10025,3041.83,2014
1,60657,1589.42,2014
2,10023,3186.67,2014
3,77494,1807.33,2014
4,60614,1786.25,2014
...,...,...,...
22696,02110,4408.57,2020
22697,20004,2505.56,2020
22698,80951,1647.88,2020
22699,11964,15800.50,2020


In [37]:
df_homes_2014_2020

,SizeRank,Zipcode,State,City,Metro,CountyName,HomePrice,Year
91101,0,10025,NY,New York,New York-Newark-Jersey City,New York County,968761.75,2014
91102,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,2014
91103,2,10023,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,2014
91104,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,2014
91105,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,2014
...,...,...,...,...,...,...,...,...
303665,34430,43523,OH,Malinta,NaN,Henry County,84331.11,2020
303666,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,579932.56,2020
303667,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,42896.56,2020
303668,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,195888.67,2020


In [38]:
#merge rent prices and home prices 2014-2020
df_zillow_merge = pd.merge(zillow_Rents2014_20, df_homes_2014_2020,how='left', on=['Year', 'Zipcode'])
df_zillow_merge

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice
0,10025,3041.83,2014,0.0,NY,New York,New York-Newark-Jersey City,New York County,968761.75
1,60657,1589.42,2014,1.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75
2,10023,3186.67,2014,2.0,NY,New York,New York-Newark-Jersey City,New York County,1024543.17
3,77494,1807.33,2014,3.0,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00
4,60614,1786.25,2014,4.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92
...,...,...,...,...,...,...,...,...,...
22696,02110,4408.57,2020,14752.0,MA,Boston,Boston-Cambridge-Newton,Suffolk County,1339232.44
22697,20004,2505.56,2020,15149.0,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,497022.00
22698,80951,1647.88,2020,15318.0,CO,Colorado Springs,Colorado Springs,El Paso County,315486.22
22699,11964,15800.50,2020,17169.0,NY,Town of Shelter Island,New York-Newark-Jersey City,Suffolk County,1015162.00


In [39]:
#look at NaN values in new data set
df_zillow_merge.isna().sum()

Zipcode        0
RentPrice      3
Year           0
SizeRank      28
State         28
City          28
Metro         28
CountyName    28
HomePrice     47
dtype: int64

In [40]:
#merge df_vacancy + FRED data 2011-2018
df_vacany_merged_fred = pd.merge(df_vacancy, df_fred_2011_2018, how='left', on='Year')
df_vacany_merged_fred

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,02333,3.024027,2.199925,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,02338,3.116343,2.948791,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,4.464646,2.066438,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
3,02341,3.586322,2.340722,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,3.732901,2.926524,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...
264955,98279,51.219512,10.993457,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264956,98280,51.329243,12.777549,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,6.540162,1.960476,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,28.537736,14.679524,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [41]:
#subset vacancy rate data for merge with zillow data
df_vacancy_2014_2018 = df_vacancy[df_vacancy.Year > 2013]

In [42]:
#change vacacny rate 2014-208 dtypes to match vacancy to prep for zillow merge
#df_vacancy_2014_2018.Zipcode = df_vacancy_2014_2018.Zipcode.astype('int')
#df_vacancy_2014_2018.dtypes

In [43]:
df_vacancy_2014_2018.dtypes

Zipcode              object
Vacancy_Rate%       float64
MOE-VacancyRate%    float64
Year                  int64
dtype: object

In [44]:
df_zillow_merge.dtypes

Zipcode        object
RentPrice     float64
Year            int64
SizeRank      float64
State          object
City           object
Metro          object
CountyName     object
HomePrice     float64
dtype: object

In [45]:
#merge df_vacancy 2014-2018 + rent, home price zillow data
df_vacancy_ZillowMerge = pd.merge(df_zillow_merge, df_vacancy_2014_2018, how='left', on=['Year', 'Zipcode'])
df_vacancy_ZillowMerge

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%
0,10025,3041.83,2014,0.0,NY,New York,New York-Newark-Jersey City,New York County,968761.75,9.011810,1.539867
1,60657,1589.42,2014,1.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,8.042922,1.343734
2,10023,3186.67,2014,2.0,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,19.964756,2.296513
3,77494,1807.33,2014,3.0,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,3.319292,1.229599
4,60614,1786.25,2014,4.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,8.468203,1.250484
...,...,...,...,...,...,...,...,...,...,...,...
22696,02110,4408.57,2020,14752.0,MA,Boston,Boston-Cambridge-Newton,Suffolk County,1339232.44,NaN,NaN
22697,20004,2505.56,2020,15149.0,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,497022.00,NaN,NaN
22698,80951,1647.88,2020,15318.0,CO,Colorado Springs,Colorado Springs,El Paso County,315486.22,NaN,NaN
22699,11964,15800.50,2020,17169.0,NY,Town of Shelter Island,New York-Newark-Jersey City,Suffolk County,1015162.00,NaN,NaN


In [46]:
#merge all prior four datasets together
df_complete = pd.merge(df_rent_merged_homes, df_vacany_merged_fred, how='left', on=['Year', 'Zipcode'])
df_complete

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,02333,1368.536,2011,8782.0,MA,East Bridgewater,Boston-Cambridge-Newton,Plymouth County,NaN,3.024027,2.199925,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17,3.116343,2.948791,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50,4.464646,2.066438,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
3,02341,1266.816,2011,10079.0,MA,Hanson,Boston-Cambridge-Newton,Plymouth County,NaN,3.586322,2.340722,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42,3.732901,2.926524,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264955,98279,1059.870,2018,23400.0,WA,Olga,NaN,San Juan County,552805.42,51.219512,10.993457,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264956,98280,993.850,2018,25265.0,WA,Eastsound,NaN,San Juan County,678499.00,51.329243,12.777549,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83,6.540162,1.960476,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17,28.537736,14.679524,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [52]:
#check NaN numbers
df_complete.isna().sum()

Zipcode                      0
RentPrice                19907
Year                         0
SizeRank                 27056
State                    27056
City                     27056
Metro                    83008
CountyName               27056
HomePrice                37378
Vacancy_Rate%                0
MOE-VacancyRate%             0
int_rate                     0
med_hIncome                  0
uspop_growth                 0
unemplt_rate                 0
newHouse_starts              0
resConstruct_spending        0
dtype: int64

In [90]:
#check % of nan in a specific row
df_vacancy_ZillowMerge.isna().sum()/len(df_vacancy_ZillowMerge)

Zipcode             0.000000
RentPrice           0.000132
Year                0.000000
SizeRank            0.001233
State               0.000925
City                0.000925
Metro               0.000925
CountyName          0.000925
HomePrice           0.002070
Vacancy_Rate%       0.286816
MOE-VacancyRate%    0.286816
dtype: float64

In [84]:
#check spelling of certain county names, metros, city, etc.

#mask = df_vacancy_ZillowMerge.CountyName.str.contains('New York')
#df_vacancy_ZillowMerge[mask == True]

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%
0,10025,3041.83,2014,0.0,NY,New York,New York-Newark-Jersey City,New York County,968761.75,9.011810,1.539867
2,10023,3186.67,2014,2.0,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,19.964756,2.296513
6,10002,2834.17,2014,6.0,NY,New York,New York-Newark-Jersey City,New York County,844740.50,7.740710,1.204937
13,10016,3199.58,2014,15.0,NY,New York,New York-Newark-Jersey City,New York County,747793.50,18.770170,2.872057
22,10009,2719.67,2014,26.0,NY,New York,New York-Newark-Jersey City,New York County,769663.08,4.795307,1.295301
...,...,...,...,...,...,...,...,...,...,...,...
22607,10280,3493.78,2020,9113.0,NY,New York,New York-Newark-Jersey City,New York County,838135.56,NaN,NaN
22655,10069,4195.78,2020,10217.0,NY,New York,New York-Newark-Jersey City,New York County,1809159.00,NaN,NaN
22669,10007,5990.11,2020,10702.0,NY,New York,New York-Newark-Jersey City,New York County,2578354.44,NaN,NaN
22683,10006,3488.44,2020,12012.0,NY,New York,New York-Newark-Jersey City,New York County,787659.78,NaN,NaN


In [74]:
#check spelling of certain county names, metros, city, etc.

#df_vacancy_ZillowMerge.CountyName.value_counts()

Los Angeles County    1162
Orange County          770
Maricopa County        756
Cook County            567
San Diego County       462
                      ... 
Monroe County            7
Carver County            7
Luzerne County           7
Saint Croix County       7
Greene County            7
Name: CountyName, Length: 269, dtype: int64

In [86]:
#look deeper into nans...

#df_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029']

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%
21,10029,2092.30,2014,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,8.524703,1.421496
3264,10029,2099.00,2015,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,7.506373,1.285644
6507,10029,2208.92,2016,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,6.679256,1.013108
9750,10029,2258.00,2017,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,6.731025,1.254007
12993,10029,2354.25,2018,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,7.089246,0.992754
16236,10029,2408.09,2019,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN
19479,10029,2421.88,2020,NaN,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN


In [91]:
#deal with specific nans
'''
df_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'State'] = 'NY'
df_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'City'] = 'New York'
df_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'Metro'] = 'New York-Newark-Jersey City'
df_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'CountyName'] = 'New York County'
'''


"\ndf_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'State'] = 'NY'\ndf_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'City'] = 'New York'\ndf_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'Metro'] = 'New York-Newark-Jersey City'\ndf_vacancy_ZillowMerge.loc[df_vacancy_ZillowMerge['Zipcode'] == '10029', 'CountyName'] = 'New York County'\n"

In [48]:
#check NaN as a percentage
df_complete.isna().sum()/len(df_complete)*100

Zipcode                   0.000000
RentPrice                 7.513210
Year                      0.000000
SizeRank                 10.211353
State                    10.211353
City                     10.211353
Metro                    31.328502
CountyName               10.211353
HomePrice                14.107035
Vacancy_Rate%             0.000000
MOE-VacancyRate%          0.000000
int_rate                  0.000000
med_hIncome               0.000000
uspop_growth              0.000000
unemplt_rate              0.000000
newHouse_starts           0.000000
resConstruct_spending     0.000000
dtype: float64

In [49]:
df_complete.describe()

,RentPrice,Year,SizeRank,HomePrice,Vacancy_Rate%,MOE-VacancyRate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
count,245053.000000,264960.000000,237904.000000,2.275820e+05,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000
mean,1067.691442,2014.500000,15646.706806,1.846688e+05,17.675088,7.996235,1.109375,60351.000000,0.682779,6.113542,1007.885417,410836.197918
std,491.626977,2.291292,9424.124602,1.858222e+05,16.437987,10.221583,0.583590,2846.855913,0.068232,1.719867,208.944873,109740.019064
min,19.960000,2011.000000,0.000000,1.042183e+04,0.000000,0.000000,0.750000,56912.000000,0.522337,3.891667,611.916667,255208.583333
25%,781.625000,2012.750000,7531.000000,8.735608e+04,6.977963,2.491948,0.750000,57756.000000,0.672832,4.741667,892.062500,321154.395833
50%,942.446000,2014.500000,15164.000000,1.340185e+05,12.799977,4.855487,0.760417,59945.500000,0.718344,5.716667,1053.500000,410493.333333
75%,1204.816000,2016.250000,23514.000000,2.141746e+05,22.580645,9.359872,1.171875,63113.750000,0.727331,7.537500,1184.291667,500871.916667
max,5620.320000,2018.000000,34430.000000,6.141946e+06,100.000000,100.000000,2.458333,64324.000000,0.730641,8.933333,1248.250000,564448.750000


### Save Data

In [92]:
df_complete.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim/master_complete_for_EDA.csv', index=False)
df_vacancy_ZillowMerge.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim/vacancy_zillowHomeRent_merge_2014_2020.csv', index=False)

### Notes


In [60]:
#need to save dataframe for 2014-2020 data (for predictions?)


#If needed later on...
#NaNs
    #find table that has state, city, metroarea, county name by zipcode to deal with thses NaNs
    #don't worry about SizeRank..??
    #rent price and home price try below... 
    #replace home price NaNs with first:
        #diff = (price of year you have) - (mean of a year you have) 
        #then with NaN do 'diff' + (mean of NaN year)

### Summary

*Note, the primary goal was the merge the separate dataframes into one dataframe from 2011-2018 (matching the dates from the ACS vacancy rate by zipcode data) that contained vacancy rate, home prices, rent prices by zipcode and the national economic data from FRED filled for all zipcodes in the corresponding year. During EDA this seemed to be less fruitful than antipcated. So as a second pass I created a data frame from 2014-2020 with home prices and rent prices from zillow and vacancy rates from ACS data.*

DATA CLEANING
- loaded data and cleaned home price dataframe:
- added leading zeros to zipcode in home prices dataframe
- subset df_homes data for year 2011-2018 (for merging with FRED data)
- subset df_homes data for year 2014-2020 (for merging with rent data)
- looked at NaNs - about 4.5% of home prices are NaN. Noted an idea on replacing home price NaNs with:
    - diff = (price of year you have) - (mean of a year you have) 
    - then replace NaN with 'diff' + (mean of NaN year)

- cleaned rent price dataframe

- cleaned FRED economic dataframe
- subset FRED data to 2011-2018 to prep for merge
- add zipcode column to 2014-2019 with zipcode '99999' which denotes National data

DATA JOINING

*See note above regarding the two dataframes*
- merge rent prices and home prices (2011-2018)
- merge rent prices and home prices 2014-2020
- merge df_vacancy + FRED data 2011-2018
- merge df_vacancy 2014-2018 + rent, home price zillow data
- merge all prior four datasets together
- inspected NaNs to deal with later if needed
- saved the 2011-2018 data frame as well as the 2014-2020 dataframe